In [25]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.models import Model

# Load your data
df = pd.read_csv("jobs_in_data.csv")

# Split the 'salary_in_usd' column by 1000
df['salary_in_usd'] /= 1000

# Define features and target variable
X_numerical = df.select_dtypes(include=np.number).drop(columns="salary_in_usd")  # Select only numeric columns
y = df["salary_in_usd"]

# Scale numerical data
scaler = StandardScaler()
X_numerical_scaled = scaler.fit_transform(X_numerical)

# Split data into train and test sets
X_train_num, X_test_num, y_train, y_test = train_test_split(X_numerical_scaled, y, test_size=0.2, random_state=42)

# Reshape the input data to include timestep dimension
X_train_reshaped = X_train_num.reshape(X_train_num.shape[0], X_train_num.shape[1], 1)
X_test_reshaped = X_test_num.reshape(X_test_num.shape[0], X_test_num.shape[1], 1)

# Define LSTM model
input_layer = Input(shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]))
lstm_layer = LSTM(128, return_sequences=True)(input_layer)
dropout_layer = Dropout(0.2)(lstm_layer)
lstm_layer2 = LSTM(64)(dropout_layer)
output_layer = Dense(1, activation='linear')(lstm_layer2)
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_reshaped, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model
y_pred = model.predict(X_test_reshaped)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", np.sqrt(mse))


Epoch 1/20
211/211 [==============================] - 10s 23ms/step - loss: 22627.5059 - val_loss: 19252.6074
Epoch 2/20
211/211 [==============================] - 3s 16ms/step - loss: 17307.1797 - val_loss: 15737.2393
Epoch 3/20
211/211 [==============================] - 3s 16ms/step - loss: 14297.7783 - val_loss: 13130.7598
Epoch 4/20
211/211 [==============================] - 3s 16ms/step - loss: 11964.2314 - val_loss: 11043.6924
Epoch 5/20
211/211 [==============================] - 3s 15ms/step - loss: 10092.5557 - val_loss: 9372.9678
Epoch 6/20
211/211 [==============================] - 3s 16ms/step - loss: 8583.7734 - val_loss: 8026.9312
Epoch 7/20
211/211 [==============================] - 3s 16ms/step - loss: 7375.5239 - val_loss: 6953.2622
Epoch 8/20
211/211 [==============================] - 3s 16ms/step - loss: 6424.6445 - val_loss: 6117.3765
Epoch 9/20
211/211 [==============================] - 3s 15ms/step - loss: 5690.9590 - val_loss: 5480.9395
Epoch 10/20
211/211 [======

Mean Squared Error: 1940.6839011141424
Root Mean Squared Error: 44.05319399446699

In [26]:
df.head()

,work_year,job_title,job_category,salary_currency,salary,salary_in_usd,employee_residence,experience_level,employment_type,work_setting,company_location,company_size
0,2023,Data DevOps Engineer,Data Engineering,EUR,88000,95.012,Germany,Mid-level,Full-time,Hybrid,Germany,L
1,2023,Data Architect,Data Architecture and Modeling,USD,186000,186.000,United States,Senior,Full-time,In-person,United States,M
2,2023,Data Architect,Data Architecture and Modeling,USD,81800,81.800,United States,Senior,Full-time,In-person,United States,M
3,2023,Data Scientist,Data Science and Research,USD,212000,212.000,United States,Senior,Full-time,In-person,United States,M
4,2023,Data Scientist,Data Science and Research,USD,93300,93.300,United States,Senior,Full-time,In-person,United States,M


In [34]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load your data
df = pd.read_csv("jobs_in_data.csv")

# Drop the "salary" column
df = df.drop(columns=["salary"])

# Split data into features and target variable
X = df.drop(columns=["salary_in_usd"])  # Features
y = df["salary_in_usd"]  # Target variable

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocess numerical data
scaler = StandardScaler()
X_train_numerical_scaled = scaler.fit_transform(X_train.select_dtypes(include=np.number))
X_test_numerical_scaled = scaler.transform(X_test.select_dtypes(include=np.number))

# Preprocess text data
max_sequence_length = 100  # Maximum sequence length for padding
X_train_text_seq = pad_sequences(X_train.select_dtypes(include=[np.object]).apply(lambda x: x.astype(str)), maxlen=max_sequence_length)
X_test_text_seq = pad_sequences(X_test.select_dtypes(include=[np.object]).apply(lambda x: x.astype(str)), maxlen=max_sequence_length)

# Define the model architecture
model = Sequential([
    Embedding(input_dim=1000, output_dim=64, input_length=max_sequence_length),
    SpatialDropout1D(0.2),
    LSTM(100),
    Dense(1)
])

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Print model summary
print("Model summary:")
print(model.summary())

# Train the model
history = model.fit([X_train_text_seq, X_train_numerical_scaled], y_train, epochs=20, batch_size=64, validation_data=([X_test_text_seq, X_test_numerical_scaled], y_test), callbacks=[EarlyStopping(patience=3)])

# Evaluate the model
y_pred = model.predict([X_test_text_seq, X_test_numerical_scaled])
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", np.sqrt(mse))


C:\Users\erga9\AppData\Local\Temp\ipykernel_4928\27342718.py:31: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  X_train_text_seq = pad_sequences(X_train.select_dtypes(include=[np.object]).apply(lambda x: x.astype(str)), maxlen=max_sequence_length)


AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations